In [23]:
# this file is to check and verify and report the result of the whole Bug Assignment:
import pandas as pd
import numpy as np
import statistics
from pandas import Timestamp

In [201]:

# multiply instead of the summation
def multiplyList(myList) :
    result = 1
    for x in myList:
         result = result * x
    return result

# get stat for -old or new and then compare
def get_stat_for(file, authors, label='-new'):
    pd.set_option("display.max_columns", None)
    df = pd.read_csv(file)
    
    df1 = pd.read_csv('../../data/output/del2/jdt_direct-21Feb.csv')
    
    
    #result = df.loc[[302470, 374920, 324237, 329614, 369558, 310687, 347302, 327079, 309163, 393007, 294322, 303155, 303794, 330680, 303163, 306752, 385989, 303945, 319689, 306763, 296781, 410830, 344552, 342249, 367465, 344436, 374264, 311162, 373885, 343422]]
    #result.to_csv('fixable1.csv')  
    
    
    count = 0
    total = 0

    previous_component = ''
    previous_time = ''
    previous = ''
    ids = set()
    for (index, row), (index_s, row_s) in zip(df.iterrows(), df1.iterrows()):
        #print(row['bug_id'] == row_s['bug_id'])
        diff = Timestamp(row['report_time']) - Timestamp(previous_time)
        hdiff = diff.total_seconds() / 3600
        
        if False and previous != '' and hdiff < 3 and row['component'] == previous_component:
            chosen = previous
        else:
            total_scores = {'history': 0, 'api': 0, 'code': 0, 'api_s': 0}
            for i in range(1, 11):
                h = row['history_at_' + str(i) + '_v']
                c = row['code_at_' + str(i) + '_v']
                a = row['api_at_' + str(i) + '_v']

                if str(h) != 'nan':
                    total_scores['history'] += h
                if str(c) != 'nan':
                    total_scores['code'] += c
                if str(a) != 'nan':
                    total_scores['api'] += a
               
                a_s = row_s['api_at_' + str(i) + '_v']
                if str(a) != 'nan':
                    total_scores['api_s'] += a_s
                    
                    
            scores = {}
            for i in range(1, 11):
                h = row['history_at_' + str(i) + '_v']
                h_developer = row['history_at_' + str(i)]
                c = row['code_at_' + str(i) + '_v']
                c_developer = row['code_at_' + str(i)]
                a = row['api_at_' + str(i) + '_v']
                a_developer = row['api_at_' + str(i)]

                a_s = row_s['api_at_' + str(i) + '_v']
                a_s_developer = row_s['api_at_' + str(i)]
                
                if str(h_developer) != 'nan':
                    if str(h) == 'nan' or h == 0 or total_scores['history'] <= 0:
                        pass
                        #scores[h_developer] = scores.get(h_developer, []) + [0]
                    else:
                        scores[h_developer] = scores.get(h_developer, []) + [float(h * 100 / total_scores['history'])]

                if str(c_developer) != 'nan':
                    if str(c) == 'nan' or c == 0 or total_scores['code'] <= 0:
                        pass
                        #scores[c_developer] = scores.get(c_developer, []) + []
                    else:
                        scores[c_developer] = scores.get(c_developer, []) + [float(c * 100 / total_scores['code'])]

                
#                 if str(a_s_developer) != 'nan':
#                     if str(a_s) == 'nan' or a_s == 0 or total_scores['api_s'] <= 0:
#                         pass
#                         #scores[a_developer] = scores.get(a_developer, []) + []
#                     else:
#                         #if row['bug_id'] not in [300805, 374920, 296589, 324237, 329614, 369558, 310687, 347302, 327079, 303785, 393007, 294322, 303155, 303794, 330680, 303163, 325694, 306752, 411841, 385989, 303945, 319689, 306763, 296781, 410830, 303313, 342249, 357997, 344436, 299127, 374264, 311162, 322556, 373885, 343422]:
#                         scores[a_s_developer] = scores.get(a_s_developer, []) + [float(a_s * 100 / total_scores['api_s'])]

          


                if str(a_developer) != 'nan':
                    if str(a) == 'nan' or a == 0 or total_scores['api'] <= 0:
                        pass
                        #scores[a_developer] = scores.get(a_developer, []) + []
                    else:
                        #if row['bug_id'] not in [302470, 374920, 324237, 329614, 369558, 310687, 347302, 327079, 309163, 393007, 294322, 303155, 303794, 330680, 303163, 306752, 385989, 303945, 319689, 306763, 296781, 410830, 344552, 342249, 367465, 344436, 374264, 311162, 373885, 343422]:
                        scores[a_developer] = scores.get(a_developer, []) + [row['jaccard_score'] * float(a * 100 / total_scores['api'])]


            for index_, score in scores.items():
                if len(score) == 0:
                    scores[index_] = 0
                else:
                    scores[index_] = sum(score)  ## ** (1/float(len(score)))


            # sorting the results
            sort = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            chosen = ''
            if len(sort) != 0:
                chosen = sort[0][0]


        if chosen == row['assignees']:
            count += 1

        previous = row['assignees']
        total += 1

        if row['assignees'] not in authors:
            authors[row['assignees']] = {
                'total-new': 0, 'count_differently-new': 0, 'count-new': 0, 'count_history-new': 0, 'count_code-new': 0, 'count_api-new': 0,
                'total-old': 0, 'count_differently-old': 0, 'count-old': 0, 'count_history-old': 0, 'count_code-old': 0, 'count_api-old': 0,
                }

        authors[row['assignees']]['total' + label] += 1
        if row['assignees'] == row['history_at_1']:
            authors[row['assignees']]['count_history' + label] += 1
        if row['assignees'] == row['code_at_1']:
            authors[row['assignees']]['count_code' + label] += 1
        if row['assignees'] == row['api_at_1']:
            authors[row['assignees']]['count_api' + label] += 1
        if 1 == row['at_1']:
            authors[row['assignees']]['count' + label] += 1
        if 0 < len(sort) and sort[0][0] == row['assignees']:
            authors[row['assignees']]['count_differently' + label] += 1
            
#         if 0 < len(sort) and row['at_1'] == 1 and 53 < sort[0][0] < 107:
#             sort[0][0]
            
            
        if 0 < len(sort) and row['at_1'] == 1 and sort[0][0] != row['assignees']:
            ids.add(row['bug_id'])
            #print(sort[0][1])
        
        previous = row['chosen']
        previous_time = row['report_time']
        previous_component = row['component']
   
    print(len(ids))
    print((ids))
    print(label)
    print(count)
    print(count / total)
    print(total)
    return authors



authors_ = {}
authors_ = get_stat_for('../../data/output/jdt_direct.csv', authors_, '-new')
#authors_ = get_stat_for('../../data/output/jdt_direct-without-util.csv', authors_, '-old')

3
{373885, 410830, 393007}
-new
270
0.45
600


In [106]:
a= 0
authors_copy = {}
for name, author in authors_.items():
    author['count_differently-change'] = author['count_differently-new'] - author['count_differently-old']
    author['count-change'] = author['count-new'] - author['count-old']
    #author['history-change'] = author['count_history-new'] - author['count_history-old']
    #author['code-change'] = author['count_code-new'] - author['count_code-old']
    author['api-change'] = author['count_api-new'] - author['count_api-old']
    #author['can-be'] = author['total-new'] - author['count_differently-new']
    a = a + author['count-change']
    
    authors_copy[name] = {'api': author['count_api-new'], 'total': author['total-new'], 'count': author['count_differently-new']}

print(a)
x = pd.DataFrame.from_dict(authors_copy)
x

266


,Dani Megert,Markus Keller,Satyam Kandula,Kent Johnson,JDT-UI-Inbox,Olivier Thomann,Deepak Azad,Frederic Fusier,JDT-Text-Inbox,Raksha Vasisht,Jayaprakash Arthanareeswaran,Paul Webster,Rajesh,Tom Hofmann,Lakshmi Shanmugam,Björn Michael,Manju Mathew,Noopur Gupta,Platform-Search-Inbox,Paul Fullbright,Ayushman Jain,Platform-Doc-Inbox,Nikolay Metchev,Szymon Brandys,Samrat Dhillon,Philippe Marschall,Stephan Herrmann,Rabea Gransberger,Manoj Palat,Snjezana Peco,Michael Rennie,Lars Vogel,Andrew Eisenberg,shankha banerjee,Marcel Bruch
api,26,136,0,0,0,0,8,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
total,145,241,2,1,8,2,85,1,6,37,2,1,7,1,1,2,18,12,1,1,2,1,3,1,5,1,4,1,1,1,2,1,1,1,1
count,26,136,0,0,0,0,8,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
a= 0
authors_copy = {}
for name, author in authors_.items():
    author['count_differently-change'] = author['count_differently-new'] - author['count_differently-old']
    author['count-change'] = author['count-new'] - author['count-old']
    #author['history-change'] = author['count_history-new'] - author['count_history-old']
    #author['code-change'] = author['count_code-new'] - author['count_code-old']
    author['api-change'] = author['count_api-new'] - author['count_api-old']
    #author['can-be'] = author['total-new'] - author['count_differently-new']
    a = a + author['count-change']
    
    authors_copy[name] = {'api': author['count_api-new'], 'total': author['total-new'], 'count': author['count_differently-new']}

print(a)
x = pd.DataFrame.from_dict(authors_copy)
x

266


,Dani Megert,Markus Keller,Satyam Kandula,Kent Johnson,JDT-UI-Inbox,Olivier Thomann,Deepak Azad,Frederic Fusier,JDT-Text-Inbox,Raksha Vasisht,Jayaprakash Arthanareeswaran,Paul Webster,Rajesh,Tom Hofmann,Lakshmi Shanmugam,Björn Michael,Manju Mathew,Noopur Gupta,Platform-Search-Inbox,Paul Fullbright,Ayushman Jain,Platform-Doc-Inbox,Nikolay Metchev,Szymon Brandys,Samrat Dhillon,Philippe Marschall,Stephan Herrmann,Rabea Gransberger,Manoj Palat,Snjezana Peco,Michael Rennie,Lars Vogel,Andrew Eisenberg,shankha banerjee,Marcel Bruch
api,39,96,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
total,145,241,2,1,8,2,85,1,6,37,2,1,7,1,1,2,18,12,1,1,2,1,3,1,5,1,4,1,1,1,2,1,1,1,1
count,39,96,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
x = pd.DataFrame.from_dict(authors_)
x

,Dani Megert,Markus Keller,Satyam Kandula,Kent Johnson,JDT-UI-Inbox,Olivier Thomann,Deepak Azad,Frederic Fusier,JDT-Text-Inbox,Raksha Vasisht,Jayaprakash Arthanareeswaran,Paul Webster,Rajesh,Tom Hofmann,Lakshmi Shanmugam,Björn Michael,Manju Mathew,Noopur Gupta,Platform-Search-Inbox,Paul Fullbright,Ayushman Jain,Platform-Doc-Inbox,Nikolay Metchev,Szymon Brandys,Samrat Dhillon,Philippe Marschall,Stephan Herrmann,Rabea Gransberger,Manoj Palat,Snjezana Peco,Michael Rennie,Lars Vogel,Andrew Eisenberg,shankha banerjee,Marcel Bruch
total-new,145,241,2,1,8,2,85,1,6,37,2,1,7,1,1,2,18,12,1,1,2,1,3,1,5,1,4,1,1,1,2,1,1,1,1
count_differently-new,77,173,0,0,0,0,10,0,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count-new,77,173,0,0,0,0,10,0,0,3,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_history-new,36,107,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_code-new,75,146,0,0,0,0,20,0,0,6,0,0,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
count_api-new,6,18,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
total-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_differently-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
count_history-old,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
